In [1]:
# !pip install datasets
from datasets import load_dataset

/Users/abindal/Desktop/Spring2024/PAN2024/Code/pan-2024/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def preprocess(text):
    # Remove newlines, tabs, and carriage returns
    text = text.replace('\n', ' ').replace('\t', ' ').replace('\r', ' ')
    # Remove extra whitespaces
    text =  ' '.join(text.split())
    # Remove 's from the text
    text = text.replace("\'s", "")
    # Remove ' from the text
    text = text.replace("\'", "")
    # Remove " from the text
    text = text.replace("\"", "")
    return text

In [3]:
def load_dataset_json(file_path):
    dataset = load_dataset('json', data_files=file_path)
    #preprocess the data
    data = dataset.map(lambda example: {'text': preprocess(example['text'])})
    return data

In [4]:
human_dataset = load_dataset_json('./dataset/pan24-generative-authorship-news/human.jsonl')

In [5]:
print(human_dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'text'],
        num_rows: 1087
    })
})


In [6]:
import os
# return a list of paths to all files in a directory
def get_all_files_in_dir(dir):
    files = []
    for root, directories, filenames in os.walk(dir):
        for filename in filenames:
            files.append(os.path.join(root, filename))
    return files

machine_dataset_paths = get_all_files_in_dir('./dataset/pan24-generative-authorship-news/machines/')
print(machine_dataset_paths)

['./dataset/pan24-generative-authorship-news/machines/text-bison-002.jsonl', './dataset/pan24-generative-authorship-news/machines/alpaca-7b.jsonl', './dataset/pan24-generative-authorship-news/machines/vicgalle-gpt2-open-instruct-v1.jsonl', './dataset/pan24-generative-authorship-news/machines/mistralai-mistral-7b-instruct-v0.2.jsonl', './dataset/pan24-generative-authorship-news/machines/meta-llama-llama-2-7b-chat-hf.jsonl', './dataset/pan24-generative-authorship-news/machines/gpt-3.5-turbo-0125.jsonl', './dataset/pan24-generative-authorship-news/machines/gpt-4-turbo-preview.jsonl', './dataset/pan24-generative-authorship-news/machines/bigscience-bloomz-7b1.jsonl', './dataset/pan24-generative-authorship-news/machines/gemini-pro.jsonl', './dataset/pan24-generative-authorship-news/machines/qwen-qwen1.5-72b-chat-8bit.jsonl', './dataset/pan24-generative-authorship-news/machines/meta-llama-llama-2-70b-chat-hf.jsonl', './dataset/pan24-generative-authorship-news/machines/mistralai-mixtral-8x7b-i

In [7]:
ai_dataset = load_dataset_json(machine_dataset_paths)

In [8]:
print(ai_dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'text'],
        num_rows: 14131
    })
})


In [9]:
import random
from datasets import Dataset
from tqdm import tqdm
import secrets
def create_new_dataset(human_dataset, ai_dataset, num_samples, batch_size=500):
    new_texts = []
    labels = []
    human_texts = human_dataset['train']["text"]
    ai_texts = ai_dataset['train']["text"]
    num_batches = num_samples // batch_size
    
    for _ in tqdm(range(num_batches)):
        batch_label = random.choice([0, 1])  # Choose the label for the batch
        
        # Select text pairs for the batch with the same label
        batch_text1 = random.sample(human_texts, batch_size)
        batch_text2 = random.sample(ai_texts, batch_size)
        
        new_texts.extend([{"id":secrets.token_hex(10), "text1": t1, "text2": t2} for t1, t2 in zip(batch_text1, batch_text2)])
        labels.extend([batch_label] * batch_size)
    
    # Shuffle the dataset
    #print("combining the dataset")
    combined_data = list(zip(new_texts, labels))
    #print("shuffling the dataset")
    random.shuffle(combined_data)
    #print("unzipping the dataset")
    new_texts, labels = zip(*combined_data)
    #print("creating the dataset")
    return Dataset.from_dict({"text_pair": new_texts, "label": labels})


In [10]:
combined_dataset = create_new_dataset(human_dataset, ai_dataset, 100000)

100%|██████████| 200/200 [00:00<00:00, 458.88it/s]


In [11]:
print(combined_dataset)

Dataset({
    features: ['text_pair', 'label'],
    num_rows: 100000
})


In [12]:
# Split the dataset into training and validation and test sets
from datasets import DatasetDict
from sklearn.model_selection import train_test_split

train_texts, test_texts, train_labels, test_labels = train_test_split(combined_dataset['text_pair'], combined_dataset['label'], test_size=0.3, random_state=42)
val_texts, test_texts, val_labels, test_labels = train_test_split(test_texts, test_labels, test_size=0.5, random_state=42)

# Save the splits into a single dataset
train_dataset = Dataset.from_dict({"text_pair": train_texts, "label": train_labels})
val_dataset = Dataset.from_dict({"text_pair": val_texts, "label": val_labels})
test_dataset = Dataset.from_dict({"text_pair": test_texts, "label": test_labels})


In [13]:
trainTestValid_dataset = DatasetDict({'train': train_dataset, 'test': test_dataset, 'valid': val_dataset})

In [14]:
trainTestValid_dataset.save_to_disk('./dataset/competition_dataset')

Saving the dataset (1/1 shards): 100%|██████████| 15000/15000 [00:00<00:00, 369301.42 examples/s]


In [15]:
#To load the dataset
# train_dataset = Dataset.load_from_disk('./dataset/competition_dataset')

In [16]:
print(trainTestValid_dataset['train'][0])

{'text_pair': {'id': '6565b9176516a1195be9', 'text1': 'Prince Harry, Meghan Markle, and Archie Will Always Hold a Special Place for Prince Philip in Their Hearts Prince Harry has returned to the UK to say goodbye to his grandfather. Over the weekend, the royal family shared that the Duke of Sussex would be flying from California to the UK for Prince Philip funeral; he is now back in the country, and staying at Frogmore Cottage, where is following COVID-19 protocols. Town & Country understands that the Duchess of Sussex made efforts to travel alongside her husband, but she did not receive clearance from her doctor as she is currently pregnant with her second child. During the Sussexes interview with Oprah earlier this spring, the couple revealed they are expecting a girl, and the baby is due to arrive sometime this summer. This morning, the Duke of Sussex shared a statement paying tribute to his grandfather in which he said: My grandfather was a man of service, honour and great humour. 